In [30]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('A3_task1_data_files/train.csv', sep='\t')
dev_df = pd.read_csv('A3_task1_data_files/dev.csv', sep='\t')

dev_df.rename(columns={'setence1': 'sentence1'}, inplace=True)

In [31]:
from torch.utils.data import DataLoader
from sentence_transformers import InputExample, datasets

# Convert the DataFrame into a list of InputExample
train_examples = [InputExample(texts=[str(row['sentence1']), str(row['sentence2'])], label=float(row['score'])) for index, row in train_df.iterrows()]
dev_examples = [InputExample(texts=[str(row['sentence1']), str(row['sentence2'])], label=float(row['score'])) for index, row in dev_df.iterrows()]

# Create a DataLoader
train_dataloader = DataLoader(train_examples, shuffle=False, batch_size=512)
dev_dataloader = DataLoader(dev_examples, shuffle=False, batch_size=512)

In [32]:
from sentence_transformers import SentenceTransformer, models, losses

# Load a pre-trained Sentence-BERT model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)

In [33]:
from sentence_transformers import evaluation

# Define an evaluator using the dev set for evaluation during training
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_examples, name='dev')

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=2,
          warmup_steps=100,
          output_path='fine-tuned-model')


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:17<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 0; 6.00 GiB total capacity; 12.05 GiB already allocated; 0 bytes free; 12.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF